In [1]:
#import relevant libaries 
from tkinter import *
from tkinter import ttk
from ttkthemes import ThemedTk
import pandas as pd
import joblib
import numpy as np 
from tkinter import messagebox
from PIL import ImageTk, Image

#categories for data entry 
Sex = 'Sex', 
fields = 'Age', 'Height', 'Weight', 'Cigarettes per day', 'Sys BP','Heart Rate','Total Cholestrol mg/dL', 'Glucose'
check = 'Blood Pressure Medication', 'Prevalent Hypertension',
education = 'Education', 


# 'const','age', 'sysBP', 'male', 'cigsPerDay', 'glucose', 'totChol', 
# 'prevalentHyp', 'education','heartRate','BMI', 'BPMeds'
def fetch(entries):
    "Function that fetches the entered data and prints it out"
    data_input = [1.0]
    data_columns = ['const']
    for entry in entries:
        field = (entry[0])
        text  = (entry[1].get())
        data_input.append(text)
        data_columns.append(field)
    order = ['const', 'Age','Sys BP', 'Sex', 'Cigarettes per day', 'Glucose', 'Total Cholestrol mg/dL',  
             'Prevalent Hypertension','Education','Heart Rate', 'Height', 'Weight', 'Blood Pressure Medication']
    df_inputs = pd.DataFrame([data_input], columns=data_columns)[order]
    df_inputs['BMI'] = float(df_inputs.Weight[0]) / (float(df_inputs.Height[0])**2)
    new_order = ['const', 'Age','Sys BP', 'Sex', 'Cigarettes per day', 'Glucose', 'Total Cholestrol mg/dL',  
             'Prevalent Hypertension','Education','Heart Rate','BMI','Blood Pressure Medication' ]
    df_inputs = df_inputs[new_order]
    
    #dictionary for education conversion
    d_edu = {
        "High school/ GCSE level": 1,
        "Sixth Form/ A level": 2,
        "Post 18 training": 3,
        "College/ University Degree": 4
        }
    
    #Convert education to int 1,2,3,4
    df_inputs['Education'].iloc[0] = d_edu[df_inputs['Education'].iloc[0]]
    
    print(df_inputs.iloc[0])
    
    if check_fields(df_inputs) == True:    
    
        #convert to list to pass to Logistic Regression to calculate % chance
        df=list(df_inputs.iloc[0]) 

        #Load scaling and fit function
        LR_jl = joblib.load('LR.pkl')
        scalar_jl = joblib.load('scaler.pkl')
        
        #Predict the probability of getting CHD within 10 years
        y = LR_jl.predict_proba(scalar_jl.transform([df_inputs.iloc[0]]))
        prob_yes = y[0,1] # the probability of prediciting yes
        messagebox.showinfo("Result", 'Probability of getting CHD is {:.2f}%:'.format(prob_yes*100))


def check_fields(df_inputs):
    '''Function to check that all user enries are valid
    - Check no missing fields
    - Check input is a number
    - Check number is within allowed range
    '''
    def is_number(s):
        "Check whether a string contains a number"
        try:
            s = float(s)
            return True        
        except ValueError: 
            return False
    
    #dictionary for parameter ranges - taken as max and min values from Framingham data - avoid extrapolation
    d_ranges = {'const':(1,1),'Age':(0,70),'Sys BP':(80,300), 'Sex':(0,1), 'Cigarettes per day':(0,70), 
                'Glucose':(40,400), 'Total Cholestrol mg/dL':(100,700), 'Prevalent Hypertension':(0,70),
                'Education':(1,4),'Heart Rate':(40,150),'BMI':(15,60),'Blood Pressure Medication':(0,1)}
    
    # Check each value is filled with a number and it is within allowed range
    for col in df_inputs.columns:
        if is_number(df_inputs[col].iloc[0]) == False: # if not integer of decimal
            messagebox.showwarning("Warning","Please input a number for {}".format(col))
            return False # error code -1
        else:
            df_inputs[col].iloc[0] = float(df_inputs[col].iloc[0])
            
        # Is value within allowed range
        if df_inputs[col].iloc[0] < d_ranges[col][0] or df_inputs[col].iloc[0] > d_ranges[col][1]:
            messagebox.showwarning("Warning","{} must be in range {}".format(col,d_ranges[col]))
            return False # error code -1
    
    return True
    
#create function to enter data 
def makeform(root, fields):
    "Function creating the entry widgets for the data, includes combo box, text entry and checkbuttons"
    entries = []
    for field in Sex:
        row = Frame(root)
        lab = Label(row, width=20, text=field, anchor='w')
        ent = IntVar()
        ent.set(1)
        rad1 = Radiobutton(row, text="Male", var=ent, value=1) # 1 for male 
        rad2 = Radiobutton(row, text="Female", var=ent, value=0) # 0 for female
        row.pack(side=TOP, fill=X, padx=5, pady=5)
        lab.pack(side=LEFT)
        rad1.pack(side=LEFT, anchor='w')
        rad2.pack(side = RIGHT, anchor ='w')
        entries.append((field, ent)) 
        
    for field in fields:
        row = Frame(root)
        lab = Label(row, width=20, text=field, anchor='w')
        ent = Entry(row)
        entries.append((field, ent))
        row.pack(side=TOP, fill=X, padx=5, pady=5)
        lab.pack(side=LEFT)
        ent.pack(side=RIGHT, expand=YES, fill=X)
    
    for field in check:
        row = Frame(root)
        lab = Label(row, width=20, text=field, anchor='w')
        ent = IntVar() 
        ent.set(0)
        chk = Checkbutton(row,  var=ent)  
        row.pack(side=TOP, fill=X, padx=5, pady=5 )
        lab.pack(side=LEFT)
        chk.pack(side=RIGHT, anchor='w')
        entries.append((field, ent)) 
    
    for field in education: 
        row = Frame(root)
        lab = Label(row, width=20, text = field, anchor ='w')
        # ent = ttk.Combobox(row, values = ['0', '1', '2', '3', '4'])
        # 1 for some high school e.g. GCSE, 2 for a 
        # high school diploma or GED e.g. A levels, 3 
        # for some college or vocational school e.g Other post , and 4 for a college degree.
        OPTIONS = [
        "High school/ GCSE level",
        "Sixth Form/ A level",
        "Post 18 training",
        "College/ University Degree"
        ]

        ent = StringVar()
        ent.set(OPTIONS[0]) # default value
        menu = OptionMenu(row, ent, *OPTIONS)
        menu.config(width=22)
        row.pack(side=TOP, fill=X, padx=5, pady=5)
        lab.pack(side=LEFT)
        menu.pack(side=RIGHT, expand=NO, fill=X)
        entries.append((field, ent))
    return entries

#quitting function to prevent crashing 
def quit(root):
    root.destroy()       
    
if __name__ == '__main__':
    "Main function to produce the interface"
    root = ThemedTk(theme = 'arc')
    root.title('CHD predictor')
    root.resizable(False, False)
    path = "logo6.png"
    img = ImageTk.PhotoImage(Image.open(path))
    panel = ttk.Label(root, image = img)
    panel.pack(side = "top", fill = "both", padx = 100, pady =(0, 20))
    panel.configure(background='white')
    ents = makeform(root, fields)   
    b1 = Button(root, text='Show',
                  command=(lambda e=ents: fetch(e)))
    b1.pack(side=RIGHT, padx=5, pady=5)
    b2 = Button(root, text="Quit", command=lambda root=root:quit(root))
    b2.pack(side=RIGHT, padx=5, pady=5)
    root.mainloop()

In [ ]:
#####
# Test that we can import the model properly
#####

#loading LR and scaling
LR_jl = joblib.load('LR.pkl')
scalar_jl = joblib.load('scaler.pkl')

# Run a test to make sure that the model and scaling are properly imported. 
# This gives the order the params should be parsed from the interface to the model
# const             1.00
# age              51.00
# sysBP           141.00
# male              0.00
# cigsPerDay        0.00
# glucose         130.00
# totChol         177.00
# prevalentHyp      1.00
# education         1.00
# heartRate        72.00
# BMI              29.64
# BPMeds            0.00
test_data = [1, 51.0, 141.0, 0.00, 0.00, 130.0, 177.0, 1.0, 1.0, 72.0, 29.64, 0.0]
prob = LR_jl.predict_proba(scalar_jl.transform([test_data]))
print('{:.2f}%'.format(100*prob[0,1]))
# should yield [0.82953316 0.17046684]

In [ ]:
import tkinter as tk

def select():
    title=var.get()
    print("selected string :"+title)
    print("corresponing integer value :"+str(choices[Dic[title]]))


choices = [1,2,3]
Dic={'title 1':0,'title 2':1,'title 3':2}
GUI = tk.Tk()
var = tk.StringVar(GUI)
var.set('title 1')
op=tk.OptionMenu(GUI, var, *Dic.keys())
op.pack(side='left',padx=20,pady=10)
bt=tk.Button(GUI,text='check value',command=select)
bt.pack(side='left',padx=20,pady=10)
GUI.mainloop()